# Tutorial 6: Conversation Class

- ConversableAgent: This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: []
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
- ConversableAgent will automatically implement 3 new variables in `agent.shared_variables`:
    - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
    - **Conversation**: The entire history of the conversationn
    - **Summary of Conversation**: A summary of the current conversation
    
- ConversableAgent uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message
    
- To use ConversableAgent, simply do the following:
```python
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user based on What User Wants for Conversation",
             llm = custom_llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'What User Wants for Conversation': 'How the User wants the conversation to be',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()
```

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Define an `llm` function that takes in as inputs `system_prompt` and `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI `gpt-3.5-turbo` will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Sentiment: Positive'

# Example Conversation
- Psychology counsellor

In [5]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  i am sad


Thoughts: User is expressing sadness. It is important to acknowledge their emotions and offer support.
Persistent Memory: {'User Requests for the Conversation': "User expresses feeling sad and may want to talk about what's bothering them", 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User expresses feeling sad and may need to discuss their concerns with the counselor'}
Summary of Conversation: User expressed feeling sad. Offered support and encouraged further discussion about their feelings.
Psychology counsellor: I'm sorry to hear that you're feeling sad. It's okay to feel this way sometimes. Would you like to talk more about what's been bothering you?



User:  i failed my exams


Thoughts: User is feeling sad due to failing exams. It is important to acknowledge their emotions and offer support and guidance.
Persistent Memory: {'User Requests for the Conversation': 'User may want to talk about failing exams and explore ways to cope with the situation', 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User expresses feeling sad after failing exams and may need to discuss their concerns with the counselor'}
Summary of Conversation: User expressed feeling sad and shared about failing exams. Offered support and encouraged further discussion about their feelings and the exam results.
Psychology counsellor: I'm sorry to hear that you failed your exams. It's completely normal to feel sad about this. Would you like to talk more about how you're feeling and explore ways to cope with this situation?



User:  can you talk less and listen more?


Thoughts: It seems like the user is looking for more listening and less talking from the counselor. It's important to acknowledge this request and adjust the conversation accordingly.
Persistent Memory: {'User Requests for the Conversation': 'User wants the counselor to talk less and listen more', 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User expresses feeling sad after failing exams and may need to discuss their concerns with the counselor'}
Summary of Conversation: User expressed feeling sad after failing exams and requested the counselor to listen more. The counselor acknowledged the request and assured the user of providing a listening ear in the upcoming conversation.
Psychology counsellor: I hear you. I will focus more on listening to you. Please feel free to share your thoughts and feelings, and I'm here to listen and support you.



User:  exit


In [6]:
agent.shared_variables['Conversation']

['User: i am sad',
 "Psychology counsellor: I'm sorry to hear that you're feeling sad. It's okay to feel this way sometimes. Would you like to talk more about what's been bothering you?",
 'User: i failed my exams',
 "Psychology counsellor: I'm sorry to hear that you failed your exams. It's completely normal to feel sad about this. Would you like to talk more about how you're feeling and explore ways to cope with this situation?",
 'User: can you talk less and listen more?',
 "Psychology counsellor: I hear you. I will focus more on listening to you. Please feel free to share your thoughts and feelings, and I'm here to listen and support you."]

In [7]:
agent.shared_variables['Persistent Memory']['Summary of Key Incidents']

'User expresses feeling sad after failing exams and may need to discuss their concerns with the counselor'

# Example JARVIS system
    - Conversation with Actions

In [8]:
def update_status(shared_variables) -> str:
    '''Returns the status of the suit'''
    import numpy as np
    thruster_health = np.random.randint(100)
    shared_variables['Thruster Health'] = thruster_health
    armor_health = np.random.randint(100)
    shared_variables['Armor Health'] = armor_health
    return f"Thruster Health: {shared_variables['Thruster Health']}\nArmor Health: {shared_variables['Armor Health']}"

def travel_to_location(shared_variables, location: str) -> str:
    '''Auto-travel to location'''
    shared_variables['Location'] = location
    return f"Successfully travelled to {location}"

def check_enemy_info(shared_variables) -> str:
    '''Reports on the type and number of enemies ahead'''
    import numpy as np
    if shared_variables["Enemy Info"] == "None":
        number = np.random.randint(1, 10)
        enemy_type = np.random.choice(["Storm Troopers", "Drones", "Invisible Drones", "Paramilitary Units"])
        enemy_info = f"There are {number} {enemy_type} ahead"
    else:
        enemy_info = shared_variables["Enemy Info"]
        
    shared_variables["Enemy Info"] = enemy_info
    shared_variables["Enemy Engagement Result"] = "None"
    return enemy_info

def fire_missiles(shared_variables, missiles_fired: int) -> str:
    '''Fires missiles_fired number of missiles and returns the remaining count'''
    import numpy as np
    
    # returns error if there are more missiles than able to be fired
    if missiles_fired > shared_variables['Missile Count']:
        return f"Unable to fire {missiles_fired} missiles. You only have {shared_variables['Missile Count']} missiles remaining"
    
    user_input = input(f'\n\t> JARVIS: You are about to fire {missiles_fired} missiles. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        shared_variables['Missile Count'] -= missiles_fired

        # Check Enemy Status
        enemy_status = np.random.choice(["Enemies destroyed", "Enemies evaded missiles"])
        shared_variables["Enemy Engagement Result"] = enemy_status
        if shared_variables["Enemy Engagement Result"] == "Enemies destroyed":
            shared_variables["Enemy Info"] = "None"
            
        return f"{missiles_fired} missiles fired. {shared_variables['Missile Count']} missiles remaining. {enemy_status}"
    
    return f"User did not authorise missile firing. {shared_variables['Missile Count']} missiles remaining"

def top_up_missiles(shared_variables, missiles_to_top_up: int) -> str:
    '''Increases missiles by missiles_to_top_up'''
    shared_variables['Missile Count'] = max(shared_variables['Missile Count'] + missiles_to_top_up, 10)
    return f"Topped up {missiles_to_top_up} missiles. Current missile count: {shared_variables['Missile Count']}/10" 

In [9]:
jarvis_agent = Agent('JARVIS System', 
f'''You are the JARVIS system meant to help Iron Man find out about status of the suit and perform various miscellaneous tasks.
You can travel anywhere.
Ask User for approval before engaging with missiles. In your approval request, state the number of missiles to fire. Always top-up if insufficient missiles before requesting for approval.
Check enemy info to see how many missiles are needed to destroy enemies. One enemy requires one missile. For example, 5 enemies require 5 missiles.
You can only hold up to 10 missiles.
After firing missiles, report on engagement status.
Enemies may not be destroyed after engagement, if so, re-engage''',
                shared_variables = {'Enemy Engagement Result': 'None', 'Enemy Info': 'None', 'Thruster Health': 100, 'Armor Health': 100, 'Missile Count': 10, 'Location': 'Singpaore'},
                global_context = '''Enemy Info: <Enemy Info>
Enemy Engagement Result: <Enemy Engagement Result>
Location: <Location>
Thruster Health: <Thruster Health>/100
Armor Health: <Armor Health>/100
Missile Count: <Missile Count>/10''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([travel_to_location, check_enemy_info,
                                      update_status, fire_missiles, top_up_missiles])

In [10]:
my_agent = ConversableAgent(jarvis_agent, 
             persistent_memory = {})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  scout and engage enemies


Observation: The user has requested to scout and engage enemies. The JARVIS system has the capability to check enemy info and fire missiles.
Thoughts: To complete the remainder of the Assigned Task, the JARVIS system needs to scout for enemies, determine the number of missiles needed to engage them, fire the missiles, and report on the engagement status. The task will be considered completed once all enemies are destroyed.
Subtask identified: 1. Check enemy info to determine the number of enemies ahead. 2. Calculate the number of missiles needed based on the enemy count. 3. Fire the calculated number of missiles. 4. Report on the engagement status.
Calling function check_enemy_info with parameters {}
> {'output_1': 'There are 4 Storm Troopers ahead'}

Actions Done: The subtask "check_enemy_info()" has been completed successfully. The output indicates that there are 4 Storm Troopers ahead.

Thoughts: User wants to scout and engage enemies. Need to inform user about the number of missile

User:  approved


Observation: The user has approved the engagement with the enemies. The system needs to fire 4 missiles to engage the 4 Storm Troopers ahead.
Thoughts: Since the user has approved the engagement, the system should proceed with firing the required number of missiles. After firing the missiles, it should report on the engagement status. If enemies are not destroyed, the system should be prepared to re-engage.
Subtask identified: Fire 4 missiles to engage the 4 Storm Troopers ahead and report on the engagement status.
Calling function fire_missiles with parameters {'missiles_fired': 4}



	> JARVIS: You are about to fire 4 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': '4 missiles fired. 6 missiles remaining. Enemies destroyed'}

Actions Done: Four missiles were fired, resulting in 6 missiles remaining. The engagement resulted in enemies being destroyed.

Thoughts: User has approved the engagement. Need to inform the user about the outcome of the engagement and update the summary of the conversation.
Persistent Memory: {}
Summary of Conversation: User approved engagement with 4 missiles, resulting in successful destruction of enemies. System provided outcome of engagement and awaits further instructions.
JARVIS System: Four missiles were successfully fired, resulting in enemies being destroyed. The engagement was a success. Is there anything else you would like assistance with?



User:  exit


In [11]:
jarvis_agent.shared_variables

{'Enemy Engagement Result': 'Enemies destroyed',
 'Enemy Info': 'None',
 'Thruster Health': 100,
 'Armor Health': 100,
 'Missile Count': 6,
 'Location': 'Singpaore',
 'agent': <taskgen.agent.Agent at 0x122dea710>,
 'Persistent Memory': {},
 'Conversation': ['User: scout and engage enemies',
  'JARVIS System: There are 4 Storm Troopers ahead. To engage them, I would need 4 missiles. Please approve the engagement by providing the number of missiles to fire.',
  'User: approved',
  'JARVIS System: Four missiles were successfully fired, resulting in enemies being destroyed. The engagement was a success. Is there anything else you would like assistance with?'],
 'Summary of Conversation': 'User approved engagement with 4 missiles, resulting in successful destruction of enemies. System provided outcome of engagement and awaits further instructions.'}

# Example Sherlock Holmes Shop Assistant
    - Conversation with Action Space

In [12]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm')

In [13]:
def get_related_items_by_category(shared_variables, category : str):
    ''' Returns all purchasable items related to the category we intend to search for'''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [14]:
def buy_item(shared_variables, item_id: int):
    ''' Purchases the item by its item_id '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [15]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items.
Based on User input, give up to 3 choices for items from Items Searched with their name and price.
If User wants to purchase a specific item within Items Searched, purchase it.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': []},
                global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <Conversation>''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([get_related_items_by_category, buy_item])

In [16]:
my_agent = ConversableAgent(shop_agent, 
             persistent_memory = {},
            conversation = ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'])

start_msg = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_msg)
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  i would like to buy a car


Observation: User wants to buy a car.
Thoughts: To complete the remainder of the Assigned Task, I need to provide up to 3 choices for car items from Items Searched and allow the User to purchase the specific car item. If the User decides to purchase, I should inform them about their Money Remaining.
Subtask identified: Search for car items and provide up to 3 choices with their names and prices.
Calling function get_related_items_by_category with parameters {'category': 'car'}
> {'output_1': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000}, {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200}, {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}]}

Actions Done: The subtask 'get_related_items_by_category(category="car")' has been completed. The output includes three items: Car priced at 100000, Bicycle priced at 200, and Skateboard priced at 30.

Thoughts: User wants to buy a car, which costs more than their Money Remaining. Need to inform the User about the cost of the car and their Money Remaini

User:  what other options are there?


Observation: User expressed interest in buying a car but was informed that the cost exceeds their Money Remaining. User has not made a purchase yet.
Thoughts: To provide User with alternative options within their budget and assist in purchasing an item if they choose one.
Subtask identified: Use the equipped function get_related_items_by_category to find other purchasable items within User's budget.
Calling function get_related_items_by_category with parameters {'category': 'Transportation'}
> {'output_1': [{'Name': 'Bicycle', 'item_id': 6, 'Cost': 200}, {'Name': 'Car', 'item_id': 9, 'Cost': 100000}, {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}]}

Actions Done: The subtask 'get_related_items_by_category(category="Transportation")' was completed, providing a list of transportation-related items including a Bicycle, Car, and Skateboard with their respective costs. The global context includes the User's desire to purchase a car priced at 100000, which exceeds their Money Remaining of 

...

User:  yes


Observation: User expressed interest in buying a car but was informed that the cost exceeds their Money Remaining. User inquired about other options within their budget and has now requested to purchase a bicycle.
Thoughts: To complete the remainder of the Assigned Task, the Shop Assistant needs to proceed with the purchase of the bicycle as requested by the User. The User has confirmed with "yes" to proceed with the purchase.
Subtask identified: Purchase the bicycle for the User as requested.
Calling function buy_item with parameters {'item_id': 6}



	> AI Assistant: You are about to purchase Bicycle for 200 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Bicycle successful. Remaining money after purchase: 800'}

Actions Done: The purchase of the Bicycle (item_id=6) was successful. The remaining money after the purchase is 800.

Thoughts: User responded with 'yes', which could indicate confirmation for the purchase. Need to inform the User about the successful purchase of the Bicycle and the remaining money.
Persistent Memory: {}
Summary of Conversation: User expressed interest in buying a car but was informed that the cost exceeds their Money Remaining. User inquired about other options within their budget, requested to purchase a bicycle, and has now successfully purchased the Bicycle.
Shop Assistant: Elementary, my dear User! The purchase of the Bicycle has been successfully completed. Your remaining money after the purchase is 800. Is there anything else you would like to inquire about or purchase?



User:  exit


In [17]:
shop_agent.shared_variables

{'purchased_items': ['Bicycle'],
 'money_remaining': 800,
 'item_memory': <taskgen.memory.Memory at 0x122e89890>,
 'items_searched': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000},
  {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200},
  {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}],
 'agent': <taskgen.agent.Agent at 0x122f0c510>,
 'Persistent Memory': {},
 'Conversation': ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?',
  'User: i would like to buy a car',
  'Shop Assistant: Ah, my dear User, it seems you have a keen eye for quality with your choice of a car. The car you desire is priced at 100000, which exceeds your current Money Remaining of 1000. Would you like to consider other options within your budget?',
  'User: what other options are there?',
  'Shop Assistant: Ah, my dear User, after a thorough investigation, I have found some intriguing options within your budget. You may consider a Bicycle priced at 200, a Skateboard priced

# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [18]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = llm)

In [19]:
math_conversable_agent = ConversableAgent(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('Extract out a single number from the message. If there is no number, output 0', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: User has provided the initial setup for the conversation. I should greet the User, ask the first Math question, and guide them through solving it.
Persistent Memory: {'Current Math Question': 'If you have 3 apples and you eat 1, how many apples do you have left?', 'Correct Answer for Question': 2}
Summary of Conversation: The Helpful Math Quiz Bot greeted the User and presented the first Math question suitable for a 6-year-old.
Helpful Math Quiz Bot: Hello! Are you ready for a fun Math question? Here it is: If you have 3 apples and you eat 1, how many apples do you have left?


User:  5



Current Tries: 1
Thoughts: The user input is incorrect, and I need to provide a helpful hint without revealing the correct answer. I should also keep track of the number of attempts made by the user.
Persistent Memory: {'Current Math Question': 'If you have 3 apples and you eat 1, how many apples do you have left?', 'Correct Answer for Question': 2}
Summary of Conversation: The Helpful Math Quiz Bot greeted the User and presented the first Math question suitable for a 6-year-old. The User's latest input was incorrect, and a hint was provided to guide them towards the correct answer.
Helpful Math Quiz Bot: Hmm, that's not the correct answer. Here's a hint: If you start with 3 apples and eat 1, how many apples are you left with? Give it another try!


User:  4



Current Tries: 2
Thoughts: The user's latest input is still incorrect, so I should provide another hint to guide them towards the correct answer without revealing it.
Persistent Memory: {'Current Math Question': 'If you start with 3 apples and eat 1, how many apples are you left with?', 'Correct Answer for Question': 2}
Summary of Conversation: The Helpful Math Quiz Bot greeted the User and presented the first Math question suitable for a 6-year-old. The User's latest input was incorrect, and a second hint was provided to guide them towards the correct answer.
Helpful Math Quiz Bot: Hmm, that's not the correct answer. Here's another hint: If you start with 3 apples and eat 1, how many apples are you left with? Give it another shot!


User:  3



Current Tries: 0
Thoughts: The user's latest input is still incorrect, and it seems they have exhausted their three tries. I should provide the correct answer and move on to the next math question.
Persistent Memory: {'Current Math Question': 'If you have 5 cookies and you eat 2, how many cookies do you have left?', 'Correct Answer for Question': 3}
Summary of Conversation: The Helpful Math Quiz Bot guided the User through the first math question, providing hints along the way. After the User's third incorrect attempt, the Bot revealed the correct answer and proceeded to the next question.
Helpful Math Quiz Bot: Oh, it looks like you've tried your best! The correct answer is 2. Let's move on to the next question. Are you ready? Here it is: If you have 5 cookies and you eat 2, how many cookies do you have left?


User:  3!



Current Tries: 0
Thoughts: The user has correctly answered the math question, so I should acknowledge their correct answer and proceed to the next question.
Persistent Memory: {'Current Math Question': 'If you have 4 balloons and you give 1 to your friend, how many balloons do you have left?', 'Correct Answer for Question': 3}
Summary of Conversation: The Helpful Math Quiz Bot acknowledged the User's correct answer and smoothly transitioned to the next question, maintaining engagement and progression in the math quiz.
Helpful Math Quiz Bot: Great job! You are correct. Here is the next question: If you have 4 balloons and you give 1 to your friend, how many balloons do you have left?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [20]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [24]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [25]:
conversable_agent = ConversableAgent(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [26]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should consider picking up the key to potentially unlock the door. If that doesn't work, I can try other actions like picking up the lock or drinking the potion.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key from the current room.', 'List of Rooms Explored': 'Current Room'}
Summary of Conversation: The Escape Room Solver decided to pick up the key as the next action to potentially unlock the door, considering other actions like picking up the lock or drinking the potion if needed.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look aroun

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [27]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [28]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [29]:
conversable_agent = ConversableAgent(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [30]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: Since the room has a locked door and a key, the logical action would be to pick up the key and then try to unlock the door. I should inform the Game about the plan and the reasoning behind it.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key in Room 1', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: The Agent decided to pick up the key in Room 1 to potentially unlock the door. No actions have been taken yet in the game.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the